# Training LeNet model on CIFAR-10

Author: Parth Patel

In [1]:
import torch
import torch.nn as nn
import pytorch_lightning as pl

import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torchinfo import summary
import torchmetrics

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize(32),
        # transforms.CenterCrop(28),
        transforms.ToTensor(),
        ]),
    'val': transforms.Compose([
        transforms.Resize(32),
        # transforms.CenterCrop(28),
        transforms.ToTensor(),
    ])
}

cifar_root = '../Datasets'
cifar_train = CIFAR10(cifar_root, train=True, download=True, transform=data_transforms['train'])
cirar_test = CIFAR10(cifar_root, train=False, download=True, transform=data_transforms['val'])
train_loader = DataLoader(cifar_train, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(cirar_test, batch_size=64, shuffle=False, num_workers=4)

img_datasets = {'train': cifar_train, 'val': cirar_test}
data_loaders = {'train': train_loader, 'val': test_loader}
dataset_sizes = {x: len(img_datasets[x]) for x in ['train', 'val']}
class_names = cifar_train.classes

Files already downloaded and verified
Files already downloaded and verified


In [3]:
from lenet import LeNet
model = LeNet(len(class_names))
summary(model, input_size=(1, 3, 32, 32))

/home/parth/Projects/venv/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Layer (type:depth-idx)                   Output Shape              Param #
LeNet                                    [1, 10]                   --
├─Sequential: 1-1                        [1, 120, 8, 8]            --
│    └─Conv2d: 2-1                       [1, 6, 32, 32]            456
│    └─ReLU: 2-2                         [1, 6, 32, 32]            --
│    └─MaxPool2d: 2-3                    [1, 6, 16, 16]            --
│    └─Conv2d: 2-4                       [1, 16, 16, 16]           2,416
│    └─ReLU: 2-5                         [1, 16, 16, 16]           --
│    └─MaxPool2d: 2-6                    [1, 16, 8, 8]             --
│    └─Conv2d: 2-7                       [1, 120, 8, 8]            48,120
│    └─ReLU: 2-8                         [1, 120, 8, 8]            --
├─Sequential: 1-2                        [1, 10]                   --
│    └─Linear: 2-9                       [1, 84]                   645,204
│    └─ReLU: 2-10                        [1, 84]                   --
│ 

In [4]:
trainer = pl.Trainer(accelerator='gpu', devices=1, max_epochs=30)
trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type       | Params
-------------------------------------------------
0 | accuracy          | Accuracy   | 0     
1 | feature_extractor | Sequential | 51.0 K
2 | classifier        | Sequential | 646 K 
-------------------------------------------------
697 K     Trainable params
0         Non-trainable params
697 K     Total params
2.788     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

2022-11-28 22:02:35.930813: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-28 22:02:36.125899: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=30` reached.


In [ ]:
# export model
torch.save(model.state_dict(), 'lenet_cifar10.pt')